In [2]:
import funcoes as fc
import numpy as np
import os
import pandas as pd
import pyautogui
import pyperclip
import time
import tkinter as tk
import undetected_chromedriver as uc
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tabulate import tabulate
from tkinter import scrolledtext
from webdriver_manager.chrome import ChromeDriverManager

In [7]:
#INSTALA O CHROME DRIVEr MAIS ATUALIZADO
# Desativa o bloqueio de pop-ups
options = webdriver.ChromeOptions()
options.add_argument("--disable-popup-blocking") 
#DEFINE O TEMPO DE EXECUÇÃO PARA CADA COMANDO DO PYAUTOGUI
pyautogui.PAUSE = 0.7
#INICIA O NAVEGADOR
# Caminho do ChromeDriver local
# chrome_driver_path = r"main\chromedriver-win64\chromedriver.exe"  # Altere para o caminho correto do seu ChromeDriver
# Configura o serviço do ChromeDriver
servico = Service(ChromeDriverManager().install())
# Inicia o navegador Chrome
navegador = webdriver.Chrome(service=servico, options=options)
navegador.maximize_window()
#ENTRA NO SEI
navegador.get('https://sei.anatel.gov.br/')


In [8]:
def get_text():
    #ENVIA TEXTO DO INPUT
    global texto_exigencia
    texto_exigencia = text_area.get("1.0", tk.END).strip()
    #FECHA A JANELA DE INPUT
    root.destroy()

#INICIALIZA A VARIAVEL LOCAL QUE RECEBERA O TEXTO DE INPUT
texto_exigencia = ""

#CRIA A JANELA DA INTERFACE PARA CAIXA DE INPUT
root = tk.Tk()
root.title("Inserir Texto de Exigência")

#INSTRUCAO DO QUE O USUARIO DEVE FAZER
label_instruction = tk.Label(root, text="Insira apenas a exigência:")
label_instruction.pack(padx=10, pady=5)

#CRIA AREA DE TEXTO COM ROLAGEM
text_area = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=80, height=20)
text_area.pack(padx=10, pady=10)

#CRIA BOTAO DE RECEBER O TEXTO
btn_get_text = tk.Button(root, text="Obter Texto", command=get_text)
btn_get_text.pack(pady=5)

root.bind('<Control-Return>', lambda event: get_text())

#INICIA O LOOP PRINCIPAL QUE MANTEM A JANELA ATIVA
root.mainloop()

In [12]:
import pygetwindow as gw

def muda_janela(janela: str):
    # Lista todas as janelas abertas
    windows = gw.getAllTitles()

    # Filtra janelas válidas (não-vazias)
    windows = [w for w in windows if w]

    for i in range(len(windows)):
        if janela in windows[i]:
            window_to_activate = i
    window = gw.getWindowsWithTitle(windows[window_to_activate])[0]
    window.activate()
    window.maximize()
# Alternar para uma janela específica
# window_to_activate = int(input("Digite o número da janela para ativar: "))
# window = gw.getWindowsWithTitle(windows[window_to_activate])[0]
# window.activate()


In [4]:
from pywinauto import Application

# Substitua pelo título ou caminho do programa que deseja manipular
app = Application().connect(title_re=".*Edge.*")  # Conecta ao Microsoft Edge
window = app.top_window() 
window.restore()  # Restaura a janela
window.set_focus()  # Traz a janela para o primeiro plano


<hwndwrapper.DialogWrapper - 'Nova guia - Trabalho — Microsoft​ Edge', Chrome_WidgetWin_1, 393416>

In [9]:
def reabre_processo():
    navegador.switch_to.default_content()
    navegador.switch_to.frame('ifrConteudoVisualizacao')
    try:
        fc.clica_noelemento(navegador, By.XPATH, "//img[contains(@src, 'svg/processo_reabrir.svg?18')]", 2)
        print('Processo foi aberto novamente')
        navegador.switch_to.default_content()
        return True
    except:
        navegador.switch_to.default_content()
        return False
def fecha_processo():
    navegador.switch_to.default_content()
    navegador.switch_to.frame('ifrConteudoVisualizacao')
    try:
        navegador.find_element(By.XPATH, "//img[contains(@src, 'svg/processo_concluir.svg?18')]").click()
        navegador.switch_to.frame('ifrVisualizacao')
        fc.clica_noelemento(navegador, By.XPATH, '//*[@id="sbmSalvar"]')
    except:
        pass
def manda_email(n_processo, codigo_rastreio, janela_principal):
    time.sleep(1)
    navegador.switch_to.default_content()
    navegador.switch_to.frame('ifrConteudoVisualizacao')
    fc.clica_noelemento(navegador, By.XPATH, "//img[contains(@src, 'svg/email_enviar.svg?18')]")
    navegador.switch_to.window(navegador.window_handles[-1])
    select_element = WebDriverWait(navegador, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="selDe"]')))
    select = Select(select_element)
    select.select_by_visible_text('ANATEL/E-mail de replicação <nao-responda@anatel.gov.br>')
    fc.endereco_email("notificacaosei.sp@anatel.gov.br", navegador)
    navegador.find_element(By.ID, 'txtAssunto').send_keys(f'Processo SEI nº {n_processo} - Aprovado ({codigo_rastreio})')
    select_element = WebDriverWait(navegador, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="selTextoPadrao"]')))
    select = Select(select_element)
    #SELECIONA O TEXTO PADRAO DE DRONES RETIDOS APROVADOS
    select.select_by_visible_text('Processo SEI ORCN - APROVAÇÃO - DRONE RETIDO')
    #ENVIA EMAIL
    navegador.find_element(By.XPATH, '//*[@id="divInfraBarraComandosInferior"]/button[1]').click()
    #FECHA ALERTA DO NAVEGADOR
    time.sleep(0.5)
    alert = Alert(navegador)
    alert.accept()
    time.sleep(2)
    # chrome=pyautogui.locateOnScreen('imagensAut/chrome.png', confidence=0.7)
    # pyautogui.click(chrome)
    navegador.switch_to.window(janela_principal)



In [8]:
processos_testar = {
                    "53500.001979/2025-25": "NM695217956BR",
                    "53500.001964/2025-67": "ND354951550BR",
                    "53500.001999/2025-04": "NM693521294BR",
                    "53500.002045/2025-19": "NM677832457BR",
                    "53500.002188/2025-12": "NM693530092BR",
                    "53500.002000/2025-36": "LB585320545HK",
                    "53500.002035/2025-75": "ND392878475BR",
                    "53500.002139/2025-80": "NM695223320BR",
                    "53500.002183/2025-90":  "ND420151147BR",
                    "53500.002261/2025-56": "AA605333196BR",
                    "53500.002273/2025-81": "NM674202545BR ",
                    "53500.002276/2025-14": "NM683240997BR",
                    "53500.002429/2025-23": "LB585351030HK",
                    "53500.002621/2025-10": "NM 696 222 056 BR",
                    "53500.002572/2025-15": "NM688042066BR",
                    "53500.002447/2025-13": "ND 401 005 566 BR",
                    "53500.002448/2025-50": "NM677652092BR",
                    "53500.002451/2025-73": "NM683086003BR",
                    "53500.002462/2025-53": "NM670237107BR",
                    "53500.002471/2025-44": "NM677757858BR",
                    "53500.002487/2025-57": "NM685165171BR",
                    "53500.002488/2025-00": "NM669110435BR",
                    "53500.002490/2025-71": "ND432804760BR",
                    "53500.002493/2025-12": "NM684341626BR",
                    "53500.002497/2025-92": "NM670483085BR",
                    "53500.002503/2025-10": "NM686106991BR",
                    "53500.002576/2025-01": "NM672066422BR",
                    "53500.002666/2025-94": "NM666546416BR",
                    "53500.002679/2025-63": "NM 671 182 489 BR",
                    "53500.002685/2025-11": "NM676496192BR",
                    "53500.002686/2025-65": "NM 669 318 228 BR",
                    "53500.002697/2025-45": "NM 695 217 409 BR",
                    "53500.002900/2025-83": "NM 684 326 941 BR",
                    "53500.002990/2025-11": "NM666438249BR",
                    "53500.002994/2025-91": "NM692008715BR",
                    "53500.002999/2025-13": "NM693892953BR",
                    "53500.003052/2025-20": "NM667449441BR",
                    "53500.003101/2025-24": "NM660466357BR",
                    "53500.002977/2025-53": "NM694962307BR",
                    "53500.003060/2025-76": "NM694143979BR",
                    "53500.003089/2025-58": "NM665872215BR",
                    "53500.003099/2025-93": "NM695223846BR",
                    "53500.003104/2025-68": "LB585309553HK",
                    "53500.003108/2025-46": "NM652884802BR",
                    "53500.003120/2025-51": "NM692816111BR",
                    "53500.003153/2025-09": "NM683471655BR",
                    "53500.003189/2025-84": "NM669363553BR",
                    "53500.003355/2025-42": "NM652458994BR",
                    "53500.003259/2025-02": "NM693761459BR",
                    "53500.003307/2025-54": "NM672755740BR",
                    "53500.003334/2025-27": "NM667519672BR",
                    "53500.003354/2025-06": "ND373827952BR",
                    "53500.003357/2025-31": "NM695788043BR",
                    "53500.003359/2025-21": "NM669325921BR",
                    "53500.003376/2025-68": "ND386461036BR",
                    "53500.003378/2025-57": "ND386461036BR",
                    "53500.003395/2025-94": "NM696246928BR",
                    "53500.003482/2025-41": "NM698138696BR",
                    "53500.003602/2025-19": "NM668006657BR",
                    "53500.003607/2025-33": "NM689037734BR",
                    "53500.003607/2025-33": "NM689037734BR",
                    "53500.003612/2025-46": "NM672173276BR",
                    "53500.003659/2025-18": "NM693521294BR",
                    "53500.003671/2025-14": "NM654697609BR",
                    "53500.003678/2025-36": "NM694861515BR",
                    "53500.003681/2025-50": "NM695984618BR",
                    "53500.003731/2025-07": "NM688780424BR",
                    "53500.003749/2025-09": "NM698305310BR",
                    "53500.003829/2025-56": "NM652387082BR",
                    "53500.003869/2025-06": "NM696562823BR",
                    "53500.003937/2025-29": "NM693457335BR",
                    "53500.003468/2025-48": "NM667371835BR",
                    "53500.003493/2025-21":" NM67109205BR",
                    "53500.003506/2025-62": "ND430773605BR",
                    "53500.003523/2025-08": "NM 700 011 890 BR",
                    "53500.003615/2025-80": "ND445604682BR",
                    "53500.003652/2025-98": "NM681545801BR",
                    "53500.003658/2025-65": "NM739004325BR",
                    "53500.003733/2025-98": "NM697332203BR",
                    "53500.003735/2025-87": "NM672671015BR",
                    "53500.003747/2025-10": "LM112815558CN",
                    "53500.003754/2025-11": "NM692112280BR",
                    "53500.003798/2025-33": "LM113007209CN",
                    "53500.003863/2025-21": "NM697595465BR",
                    "53500.003866/2025-64": "NM693750460BR",
                    "53500.003881/2025-11": "ND451954974BR",
                    "53500.003891/2025-48": "NM697218645BR",
                    "53500.004077/2025-41": "CNBR00039909888",
                    "53500.004227/2025-16": "NM655587896BR",
                    "53500.004600/2025-39": "NM694852315BR",
                    "53500.004591/2025-86": "ND434587379BR",
                    "53500.004583/2025-30": "NM666962249BR",
                    "53500.004559/2025-09": "NM688316114BR",
                    "53500.004230/2025-30": "NM738925075BR",
                    "53500.004232/2025-29": "NM738929236BR",
                    "53500.004236/2025-15": "NM699053325BR",
                    "53500.004238/2025-04": "NM738922710BR",
                    "53500.004241/2025-10": "NM699043694BR",
                    "53500.004259/2025-11": "ND481442846BR",
                    "53500.004324/2025-17": "NM699041146BR",
                    "53500.004342/2025-91": "NM689547114BR",
                    "53500.004402/2025-75": "NM661841703BR",
                    "53500.004431/2025-37": "ND 431 701 097 BR",
                    "53500.004451/2025-16": "NM 671 418 785 BR",
                    "53500.004556/2025-67": "NM697873212BR",
                    "53500.004558/2025-56": "NM6987404679BR",
                    "53500.004570/2025-61": "NM664176806BR",
                    "53500.004573/2025-02": "LM112807976CN",
                    "53500.004589/2025-15": "ND433581658BR",
                    "53500.004590/2025-31": "NM690568672BR",
                    "53500.004679/2025-06": "NM671407791BR",
                    "53500.004717/2025-12": "LM112445282CN",
                    "53500.004729/2025-47": "NM742213535BR",
                    "53500.004734/2025-50": "NM740063498BR",
                    "53500.004756/2025-10": "NM695991208BR",
                    "53500.004769/2025-99": "NM739118384BR",
                    "53500.004774/2025-00": "ND451465993BR",
                    "53500.004791/2025-39": "ND451465993BR",
                    "53500.004965/2025-63": "LM113006526CN",
                    "53500.004963/2025-74": "NM738943900BR"}


In [10]:
janela_principal = navegador.current_window_handle

for processo, codigo_rastreio in processos_testar.items():
    print('\nProcessando processo', processo)
    fc.vai_para_processo(navegador, processo)
    processo_reaberto = reabre_processo()
    navegador.switch_to.frame('ifrArvore')
    fc.clica_noelemento(navegador, By.PARTIAL_LINK_TEXT, "Despacho Decisório")
    manda_email(processo, codigo_rastreio, janela_principal)
    fecha_processo()




Processando processo 53500.001979/2025-25

Processando processo 53500.001964/2025-67

Processando processo 53500.001999/2025-04

Processando processo 53500.002045/2025-19

Processando processo 53500.002188/2025-12

Processando processo 53500.002000/2025-36

Processando processo 53500.002035/2025-75

Processando processo 53500.002139/2025-80

Processando processo 53500.002183/2025-90

Processando processo 53500.002261/2025-56

Processando processo 53500.002273/2025-81

Processando processo 53500.002276/2025-14

Processando processo 53500.002429/2025-23

Processando processo 53500.002621/2025-10

Processando processo 53500.002572/2025-15

Processando processo 53500.002447/2025-13

Processando processo 53500.002448/2025-50

Processando processo 53500.002451/2025-73

Processando processo 53500.002462/2025-53

Processando processo 53500.002471/2025-44

Processando processo 53500.002487/2025-57

Processando processo 53500.002488/2025-00

Processando processo 53500.002490/2025-71
Processo fo

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x002874A3+25091]
	(No symbol) [0x0020DC04]
	(No symbol) [0x000EB373]
	(No symbol) [0x0012F4DC]
	(No symbol) [0x0012F65B]
	(No symbol) [0x0016D8E2]
	(No symbol) [0x00151F54]
	(No symbol) [0x0016B49E]
	(No symbol) [0x00151CA6]
	(No symbol) [0x001231D5]
	(No symbol) [0x0012435D]
	GetHandleVerifier [0x005807C3+3142947]
	GetHandleVerifier [0x00591A2B+3213195]
	GetHandleVerifier [0x0058C412+3191154]
	GetHandleVerifier [0x00328720+685184]
	(No symbol) [0x00216E1D]
	(No symbol) [0x00213E18]
	(No symbol) [0x00213FB6]
	(No symbol) [0x002066F0]
	BaseThreadInitThunk [0x76827BA9+25]
	RtlInitializeExceptionChain [0x7750C0CB+107]
	RtlClearBits [0x7750C04F+191]
	(No symbol) [0x00000000]


In [20]:
navegador.switch_to.window(navegador.window_handles[0])

In [27]:
pyautogui.hotkey('win', 'l')

In [5]:
def abrir_janela():
    janela = tk.Toplevel()
    janela.title("Nova Janela")
    janela.geometry("300x200")

    # Garante que a nova janela fique em foco e no topo
    janela.attributes('-topmost', 1)  # Mantém a janela no topo
    janela.lift()                     # Eleva a janela
    janela.focus_force()              # Força o foco na janela

    tk.Label(janela, text="Essa janela está em foco!").pack(pady=20)
    tk.Button(janela, text="Fechar", command=janela.destroy).pack()

# Janela principal
root = tk.Tk()
root.title("Janela Principal")
root.geometry("300x200")

tk.Button(root, text="Abrir Nova Janela", command=abrir_janela).pack(pady=50)

root.mainloop()

In [30]:
if processo_reaberto:
    fecha_processo()

In [14]:

navegador.switch_to.default_content()
navegador.switch_to.frame('ifrArvore')
lista_despachos = navegador.find_elements(By.PARTIAL_LINK_TEXT, 'Despacho Decisório')
for i in lista_despachos:
    navegador.switch_to.default_content()
    navegador.switch_to.frame('ifrArvore')
    i.click()
    navegador.switch_to.default_content()
    navegador.switch_to.frame('ifrConteudoVisualizacao')
    navegador.switch_to.frame('ifrVisualizacao')
    time.sleep(1)
    if fc.check_element_exists(By.XPATH, "//*[contains(text(), 'Revogar o')]", navegador):
        navegador.switch_to.default_content()
        navegador.switch_to.frame('ifrConteudoVisualizacao')
        print('processo achado')
        time.sleep(1)
        #CLICA NO ICONE DE LEGO
        fc.clica_noelemento(navegador, By.XPATH, "//img[contains(@src, 'svg/bloco_incluir_protocolo.svg?18')]")
        
        #navegador.find_element(By.XPATH, '//*[@id="divArvoreAcoes"]/a[8]').click()
        #SELECIONA BLOCO (SELECIONA O PRIMEIRO DESPACHO PARA DRONES APROVADOS QUE LER)
        time.sleep(1.5)
        navegador.switch_to.frame('ifrVisualizacao')
        select_element = navegador.find_element(By.ID, 'selBloco')
        select = Select(select_element)
        #PROCURA O PRIMEIRO BLOCO QUE TENHA O TEXTO "Despachos para Drones aprovados"
        for opcao in select.options:
            if "Despachos para Drones aprovados" in opcao.text:
                select.select_by_visible_text(opcao.text)
                break
        time.sleep(0.5)
        #CLICA NO BOTAO DE INCLUIR NO BLOCO
        fc.clica_noelemento(navegador, By.XPATH,'//*[@id="sbmIncluir"]')
        #navegador.find_element(By.XPATH, '//*[@id="sbmIncluir"]').click()
        



processo achado


In [15]:
#VOLTA PARA PAGINA INICIAL DO PROCESSO
navegador.switch_to.default_content()
navegador.find_element(By.ID,'txtPesquisaRapida').send_keys('53500.047484/2024-61')
elementos = navegador.find_element(By.ID,'txtPesquisaRapida')
elementos.send_keys(Keys.ENTER)
navegador.switch_to.frame('ifrConteudoVisualizacao')
time.sleep(1)
#ADICIONA NOTA PARA AGUARDAR ASSINATURA
#CLICA NO ICONE DE ANOTACAO
fc.clica_noelemento(navegador, By.XPATH, "//img[contains(@src, 'svg/anotacao_cadastro.svg?18')]")
#clica_noelemento(navegador, By.XPATH,'//*[@id="divArvoreAcoes"]/a[16]')
time.sleep(0.1)
#INSERE O TEXTO DA ANOTACAO
navegador.switch_to.frame('ifrVisualizacao')
navegador.find_element(By.ID, 'txaDescricao').send_keys('Pedido de Cancelamento')
#SALVA O TEXTO
navegador.find_element(By.NAME, 'sbmRegistrarAnotacao').click()
#DA COMO APROVADO E ANOTA NO TXT DE PROCESSOS CONFORMES

In [2]:
import funcoes as fc
fc.preenche_planilhageral('53500.000742/2025-27', 'André Jacinto Rodrigues', 'Não', 'Aprovado', '', 'Rafael Basilio Chaves')

Célula encontrada: 53500.000742/2025-27
Parece que a janela 'Google Chrome' não está visível para o programa.
Parece que a janela 'theomation' não está visível para o programa.
